In [2]:
import os
import sys
import datetime
import torch
import pandas as pd
import time
import copy
import torchvision.models as models
import torch.optim as optim
import torch.nn as nn

from sklearn.model_selection import train_test_split
from tqdm import tqdm
from pytorch_lightning.metrics.classification import F1

In [3]:
data_root = "../dataset"
images_root = os.path.join(data_root, "images_all")

train_data_dist = os.path.join(data_root, "train_data.csv")

In [4]:
scripts_path = "../scripts"

In [5]:
sys.path.append(scripts_path)

In [9]:
import constants as const

from data_loader import MelanomaClassificationDataset
from cls_train_utils import get_data_loader

### Load dataset for classification problem

In [10]:
train_data = pd.read_csv(train_data_dist)

In [11]:
train_data.head()

,name,class
0,ISIC_0014745.png,benign
1,ISIC_0000757.png,benign
2,ISIC_0009256.png,benign
3,ISIC_0012967.png,benign
4,ISIC_0012089.png,benign


Process the data to transform benign/malignant into 0/1

In [12]:
train_data_tr = train_data.copy()
train_data_tr = train_data_tr.replace({"class": {"benign": 0, "malignant": 1}})

In [13]:
train_data_tr.head()

,name,class
0,ISIC_0014745.png,0
1,ISIC_0000757.png,0
2,ISIC_0009256.png,0
3,ISIC_0012967.png,0
4,ISIC_0012089.png,0


In [14]:
print("We have {} benign data points".format(len(train_data_tr[train_data_tr["class"] == 0])))
print("We have {} malignant data points".format(len(train_data_tr[train_data_tr["class"] == 1])))

We have 4759 benign data points
We have 779 malignant data points


### Create a Data Loader

In [15]:
device = const.DEVICE
classes = const.CL_CLASSES

In [27]:
const.train_val_split_size

0.75

In [28]:
train_data_split, val_data_split = train_test_split(train_data_tr, 
                                                    random_state = const.random_seed_const, 
                                                    train_size = 0.8,
                                                    shuffle = True)
    
train_dataset = MelanomaClassificationDataset(csv_file = train_data_split, 
                                              root_dir = images_root,
                                              augmentation = MelanomaClassificationDataset.get_default_transformation(),
                                              preprocessing = MelanomaClassificationDataset.get_default_preprocessing())


validation_dataset = MelanomaClassificationDataset(csv_file = val_data_split, 
                                                   root_dir = images_root,
                                                   preprocessing = MelanomaClassificationDataset.get_default_preprocessing())

In [29]:
print("We have {} benign data points in train set".format(len(train_data_split[train_data_split["class"] == 0])))
print("We have {} malignant data points in train set".format(len(train_data_split[train_data_split["class"] == 1])))
print("Fraction 1:{:.1f}".format(len(train_data_split[train_data_split["class"] == 0]) / len(train_data_split[train_data_split["class"] == 1])))

We have 3788 benign data points in train set
We have 642 malignant data points in train set
Fraction 1:5.9


In [30]:
print("We have {} benign data points in val set".format(len(val_data_split[val_data_split["class"] == 0])))
print("We have {} malignant data points in val set".format(len(val_data_split[val_data_split["class"] == 1])))
print("Fraction 1:{:.1f}".format(len(val_data_split[val_data_split["class"] == 0]) / len(val_data_split[val_data_split["class"] == 1])))

We have 971 benign data points in val set
We have 137 malignant data points in val set
Fraction 1:7.1


In [23]:
num_samples_benign = len(train_data_split[train_data_split["class"] == 0]) 
num_samples_malignant = len(train_data_split[train_data_split["class"] == 1]) 
num_samples_general = [num_samples_benign, num_samples_malignant]

normed_weights = [1 - (x / sum(num_samples_general)) for x in num_samples_general]
normed_weights = torch.FloatTensor(normed_weights).to(device)
print(normed_weights)

tensor([0.1401, 0.8599], device='cuda:0')


In [19]:
train_loader = get_data_loader(train_dataset, batch_size = const.batch_size_train, shuffle=True, num_workers = 0)
val_loader = get_data_loader(validation_dataset, batch_size = const.batch_size_val, shuffle=False, num_workers = 0)

In [20]:
data_loaders_dict = {"train": train_loader, "val": val_loader}

In [21]:
dataset_sizes = {"train": train_dataset.__len__(), "val": validation_dataset.__len__()}
class_names = classes

### Create a model

In [1]:
def generate_model_name(architecture: str = "resnet-34"):
    return "{}_{}".format(architecture, datetime.datetime.now())

In [22]:
metric = F1(num_classes = 2)

In [27]:
def train_model(model, dataloaders, criterion, optimizer, scheduler, architecture, num_epochs=25):
    since = time.time()
    model_name = '../models/{}.pth'.format(generate_model_name(architecture))
    print(model_name)

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch + 1, num_epochs))
        print('-' * 50)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0
            running_f1 = 0.0

            # Iterate over data.
            print("{} phase".format(phase))
            for inputs, labels in tqdm(dataloaders[phase]):
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)
                    f1 = metric(outputs.cpu(), labels.cpu())
                    running_f1 += f1
                    

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                
            
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]
            
            if phase == 'train':
                epoch_f1 = running_f1 / (dataset_sizes[phase]  / 8)
            else:
                epoch_f1 = running_f1 / dataset_sizes[phase]

            print('{} Loss: {:.4f} F1: {:.4f}'.format(
                phase, epoch_loss, epoch_f1))

            # Deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                torch.save(model, model_name)
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [28]:
# Number of classes in the dataset
num_classes = len(const.CL_CLASSES)

model_ft = models.inception_v3(pretrained=True)
model_ft.aux_logits=False
num_ftrs = model_ft.fc.in_features

model_ft.fc = nn.Linear(num_ftrs, len(const.CL_CLASSES))

model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss(normed_weights)
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)
exp_lr_scheduler = optim.lr_scheduler.StepLR(optimizer_ft, step_size=5, gamma=0.1)

In [29]:
model_ft = train_model(model_ft, data_loaders_dict, criterion, 
                       optimizer_ft, exp_lr_scheduler, architecture = "inception_v3", num_epochs=25)

  0%|          | 0/520 [00:00<?, ?it/s]

../models/inception_v3_2021-03-24 14:50:26.502880.pth
Epoch 1/25
--------------------------------------------------
train phase


  0%|          | 2/1385 [00:00<01:44, 13.25it/s]

train Loss: 0.5860 F1: 0.7320
val phase


100%|██████████| 1385/1385 [01:41<00:00, 13.70it/s]


...

train Loss: 0.5193 F1: 0.7590
val phase


  0%|          | 0/520 [00:00<?, ?it/s]

val Loss: 0.4207 F1: 0.7430

Epoch 3/25
--------------------------------------------------
train phase


 16%|█▌        | 84/520 [01:27<07:36,  1.05s/it]


KeyboardInterrupt: 